# **HW2: Decision Tree** 
In *assignment 2*, you need to:

1.   Implement the decision tree in 3 steps (with the example data)


*   Step 1: calculate the entropy
*   Step 2: search for the best split
*   Step 3: build the decision tree

2.   Predict the patients' death (hospDIED) in the *MIMIC* dataset

Please fill in your **studentID** here.

In [1]:
STUDENT_ID = '106062304'

# **1. Implement the Decision Tree**
In the first part, you need to implement the decision tree by completing the given funcitions.

Also, you need to run those functions with given input variables and save the output to the implementation csv file **[STUDENT_ID]_implementation.csv**.

Implement a binary decision tree to classify *Restaurant* dataset.

## Import Packages

Note: You **cannot** import any other packages in the first part (implementation)!

In [2]:
import numpy as np
import pandas as pd
import random
# import ssl

# ssl._create_default_https_context = ssl._create_unverified_context

## Load the Example Data
First, load the *Restaurant waiting* dataset: **data.csv**

In [86]:
example = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/data.csv')
example

,Unnamed: 0,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
0,X1,T,F,F,T,Some,High,F,T,French,8,T
1,X2,T,F,F,T,Full,Low,F,F,Thai,40,F
2,X3,F,T,F,F,Some,Low,F,F,Burger,8,T
3,X4,T,F,T,T,Full,Low,F,F,Thai,12,T
4,X5,T,F,T,F,Full,High,F,T,French,70,F
5,X6,F,T,F,T,Some,Medium,T,T,Italian,3,T
6,X7,F,T,F,F,None,Low,T,F,Burger,7,F
7,X8,F,F,F,T,Some,Medium,T,T,Thai,6,T
8,X9,F,T,T,F,Full,Low,T,F,Burger,80,F
9,X10,T,T,T,T,Full,High,F,T,Italian,20,F


In [87]:
example = example.drop(['Unnamed: 0'], axis=1)

In [88]:
# change the string categorical to integer labels
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

example['Alternate'] = labelencoder.fit_transform(example['Alternate'])
example['Bar'] = labelencoder.fit_transform(example['Bar'])
example['Friday'] = labelencoder.fit_transform(example['Friday'])
example['Hungry'] = labelencoder.fit_transform(example['Hungry'])
example['Patrons'] = labelencoder.fit_transform(example['Patrons'])
example['Price'] = labelencoder.fit_transform(example['Price'])
example['Raining'] = labelencoder.fit_transform(example['Raining'])
example['Reservation'] = labelencoder.fit_transform(example['Reservation'])
example['Type'] = labelencoder.fit_transform(example['Type'])

example

,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
0,1,0,0,1,2,0,0,1,1,8,T
1,1,0,0,1,0,1,0,0,3,40,F
2,0,1,0,0,2,1,0,0,0,8,T
3,1,0,1,1,0,1,0,0,3,12,T
4,1,0,1,0,0,0,0,1,1,70,F
5,0,1,0,1,2,2,1,1,2,3,T
6,0,1,0,0,1,1,1,0,0,7,F
7,0,0,0,1,2,2,1,1,3,6,T
8,0,1,1,0,0,1,1,0,0,80,F
9,1,1,1,1,0,0,0,1,2,20,F


In [89]:
example.columns

Index(['Alternate', 'Bar', 'Friday', 'Hungry', 'Patrons', 'Price', 'Raining',
       'Reservation', 'Type', 'WaitEstimate', 'Wait'],
      dtype='object')

## Calculating the Entropy *(20%)*


In [90]:
# split the data by given attribute and its threshold
def partition(data, column, threshold):
    
#   """
#   The *partition* function will split the input data into 2 branches.
#     args:
#     *   data(DataFrame): the input data
#     *   column(str): the attribute(column name)
#     *   threshold(float): the column's threshold for splitting the data
#     returns:
#     *   match_branch(DataFrame): the divided data that matches the assigned column's threshold
#     *   false_branch(DataFrame): the divided data that doesn't match the assigned column's threshold
#   """

        match_branch =data[data[column] <= threshold]
        false_branch = data[data[column] > threshold]


        return match_branch, false_branch

If we try *partition(example, 'Patrons', 1.5)*:

split example to 

match_left(Patrons<= 1.5) (Patrons != Some) and  
false_right(Patrons > 1.5) (Patrons == Some)

column: Patrons, threshold: 1.5



In [91]:
match_left, false_right = partition(example, 'Patrons', 1.5)
print(len(match_left))
print(false_right.shape)
print(len(example))
false_right

8
(4, 11)
12


,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
0,1,0,0,1,2,0,0,1,1,8,T
2,0,1,0,0,2,1,0,0,0,8,T
5,0,1,0,1,2,2,1,1,2,3,T
7,0,0,0,1,2,2,1,1,3,6,T


In [92]:
def entropy(data):
    n = len(data)
    match = data[data['Wait']=='T']
    if(len(match)==n):
        return 0
    if(len(match)==0):
        return 0
    true = len(match)/n
    false = (n-len(match))/n
    t = np.log2(true)
    f = np.log2(false)
#   """
#   The *entropy* function will calculate the entropy of the node(data)
#   args:
#   *    data(DataFrame): the data you're calculating for the entropy
#   output:
#   *    entropy(float): the node(data)'s entropy     
#   """
    entropy = 0-(true*t)-(false*f)

    return entropy

In [93]:
match, false = partition(example, 'Hungry', 0.5)
entropy1 = entropy(match)
print(entropy1)
# 0.7219280948873623

0.7219280948873623


In [94]:
match

,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
2,0,1,0,0,2,1,0,0,0,8,T
4,1,0,1,0,0,0,0,1,1,70,F
6,0,1,0,0,1,1,1,0,0,7,F
8,0,1,1,0,0,1,1,0,0,80,F
10,0,0,0,0,1,1,0,0,3,8,F


In [95]:
false

,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
0,1,0,0,1,2,0,0,1,1,8,T
1,1,0,0,1,0,1,0,0,3,40,F
3,1,0,1,1,0,1,0,0,3,12,T
5,0,1,0,1,2,2,1,1,2,3,T
7,0,0,0,1,2,2,1,1,3,6,T
9,1,1,1,1,0,0,0,1,2,20,F
11,1,1,1,1,0,1,0,0,0,40,T


An example of the output from entropy (Use this to check if your output is correct): 

In [96]:
#implementation 1: calculate the entropy of 'false' data
ans_entropy = entropy(false)

In [97]:
ans_entropy

0.863120568566631

In [98]:
entropy(false_right)

0

In [99]:
a = np.sort(example['WaitEstimate'])
b = np.unique(a)
b

array([ 3,  6,  7,  8, 12, 20, 40, 70, 80])

In [100]:
print(b[0])
len(b)

3


9

In [101]:
def mean(b):
    map = []
    for i in range(len(b)-1):
        m = (b[i]+b[i+1])/2
        map.append(m)
    return map

## Find the Best Split *(20%)*

In [102]:
# search for the best attribute and the value(threshold) to split the data
def findBestSplit(data):
#     """
#     The *findBestSplit* function finds the best combination of attribute and value(with the largest reduction in entropy) to split the data.
#     args:
#     *   data(DataFrame): the data you try to split(build the decision tree)
#     output:
#     *   column_best(str): the attribute(column) split with the largest reduction in entropy
#     *   value_best(float): the value(threshold) of the column_best attribute to split the data
#     """
    all_entropy = 9999        #the overall entropy : save the value of the smallest entropy
    column_best = 0
    value_best = 0
    att = data.drop(['Wait'],axis=1)
    
    for item in att.columns:
            a = []
            a = np.sort(data[item])
            b = np.unique(a)
            th = mean(b)
            for i in th:
                match, false = partition(data ,item, i)
                olden = entropy(data)
                en = entropy(false)
                leften = entropy(match)
#                 print(item)
#                 print(i)
# #                 print(len(data))
# #                 print(len(data[data['Wait']=='T']))
# #                 print(len(data[data['Wait']=='F']))
#                 print(len(match))
#                 print(len(false))
# # #                 print(olden)
#                 print(leften)
#                 print(en)
#                 ig = en
                ig = (len(match)/len(data))*leften+(len(false)/len(data))*en
#                 print(ig)
                if(ig<all_entropy):
                    all_entropy = ig
                    column_best = item
                    value_best = i
            
    
    return column_best, value_best
  

An example of the output from findBestSplit (Use this to check if your output is correct):

In [103]:
column_best, value_best = findBestSplit(example)
print(column_best)
print(value_best)

Patrons
1.5


In [104]:
false

,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
0,1,0,0,1,2,0,0,1,1,8,T
1,1,0,0,1,0,1,0,0,3,40,F
3,1,0,1,1,0,1,0,0,3,12,T
5,0,1,0,1,2,2,1,1,2,3,T
7,0,0,0,1,2,2,1,1,3,6,T
9,1,1,1,1,0,0,0,1,2,20,F
11,1,1,1,1,0,1,0,0,0,40,T


In [105]:
#implementation 2: Find the best split of the 'false' data
ans_column, ans_value = findBestSplit(false)

In [106]:
print(ans_column)
print(ans_value)

WaitEstimate
16.0


## Decision Tree Building *(30%)*

Use the above functions to help building the decision tree





In [107]:
def buildTree(df, depth):
#   """
#   The *buildTree* function builds the decision tree
#   args:
#   *     df(DataFrame): the data you want to apply the decision tree
#   *     depth(int) : the depth of your tree
#   output:
#   *     decisionSubTree(dict): the decision tree structure including root, branch, leaf(with the attributes and thresholds)
#   *     features(list): the features(attributes) name in the decision tree structure(from root to branch and leaf)
#   *     thresholds(list): the corresponding thresholds for the features in the 'features' list
#   """
    while(depth>0):
        column_best, value_best = findBestSplit(df)
        left,right = partition(df , column_best, value_best)
        print('run')
        features.append(column_best)
        thresholds.append(value_best)
        decisionSubTree.append((entropy(left),column_best,entropy(right)))
        if(entropy(left)>0):
            buildTree(left,depth-1)
        if(entropy(right)>0):
            buildTree(right,depth-1)
        
        return decisionSubTree, features, thresholds
  

An example of the output from buildTree (Use this to check if your output is correct): 

In [108]:
decisionSubTree = []
features= []
thresholds = []
tree,features, thresholds= buildTree(example, 10)
print(tree)
print(features)
print(thresholds)

run
run
run
run
[(0.8112781244591328, 'Patrons', 0), (0, 'Hungry', 1.0), (0, 'Friday', 0.9182958340544896), (0, 'Price', 0)]
['Patrons', 'Hungry', 'Friday', 'Price']
[1.5, 0.5, 0.5, 0.5]


In [109]:
#implementation 3: decision tree building (depth=10)
decisionSubTree = []
features= []
thresholds = []
ans_tree, ans_features, ans_thresholds = buildTree(example,10)

run
run
run
run


In [110]:
#save implementation csv
ans_path = STUDENT_ID + '_implementation.csv'

imp = []
imp.append(ans_entropy)
imp.append(ans_column)
imp.append(ans_value)
for i in range(len(ans_features)):
    imp.append(ans_features[i])
for m in range(len(ans_thresholds)):
    imp.append(ans_thresholds[m])
print(imp)
pd.DataFrame(imp).to_csv(ans_path, header = None, index = None)

[0.863120568566631, 'WaitEstimate', 16.0, 'Patrons', 'Hungry', 'Friday', 'Price', 1.5, 0.5, 0.5, 0.5]


# **2. Classification with the MIMIC Dataset**
In the second part, you need to classify **'hospDIED'(death)** in the MIMIC dataset.

Please put the classification result in a csv file. (**[STUDENTID]_prediction.csv**)

**Note:** Decision tree is recommended but not mandatory.

## Prediction (Performance) *(20%)*

The **y_test** (hospDIED) of this data is hidden, you need to use the x_test.csv to predict the y_test.

**Note:** You **can** now import the packages you need here!

Get the MIMIC data here!

Data Description: 

[Data Description](https://docs.google.com/spreadsheets/d/1pxqxQFhIcv_hrgWEtwhXE6zBVQ5ISa-13PIhvXMtWCY/edit?usp=sharing) (You can find the data description here.)

**Note:**
*   You can select the features you want to use.
*   You can use any ML models to predict the y_test.

  (However, there is a 10% bonus if you **visualize** the decision tree in this part)


[ref]: *Johnson, A. E. W., Pollard, T. J., Shen, L., Lehman, L. H., Feng, M., Ghassemi, M., Moody, B., Szolovits, P., Celi, L. A., & Mark, R. G. (2016). MIMIC-III, a freely accessible critical care database. Scientific Data, 3, 160035.*


In [111]:
#Read data
x_train = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/x_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/y_train.csv')
x_train = x_train.drop(['indextime'], axis=1)
# x_train = x_train.drop(['subject_id'], axis=1)
x_train.head()
# x_train['hospDIED']

,subject_id,age,los,CA,DNR,CMO,DNI,female,first_careunit,insurance,...,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10246985,48,1.826250,0,0,0,0,1,10,1,...,2,9,9,9,9,0,9,9,0,9
1,14538806,59,10.581123,0,0,0,0,1,10,1,...,4,4,4,4,4,4,4,4,4,4
2,12850130,73,2.065255,0,0,0,0,1,4,1,...,4,4,4,4,4,4,4,4,4,4
3,11810761,84,1.435428,0,0,0,0,1,10,3,...,0,9,9,0,9,0,9,1,9,9
4,13109236,66,1.443414,0,0,0,0,0,1,1,...,4,4,4,4,4,4,4,4,4,4


Complete your model with validation:

In [112]:
#Split the data into training and validation sets
#Build you model and evaluate it on your validation set
from sklearn.model_selection import train_test_split

x = x_train
y = y_train
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=0)

Make the final prediction from your model:

In [113]:
y_test

,hospDIED
18963,0
11813,0
10510,0
11937,0
11753,0
...,...
3033,0
4006,0
657,1
2351,0


In [114]:
x_test

,subject_id,age,los,CA,DNR,CMO,DNI,female,first_careunit,insurance,...,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
18963,15989658,70,1.083912,0,0,0,0,0,1,3,...,4,4,4,4,4,4,4,4,4,4
11813,12834999,89,1.767269,0,0,0,0,0,10,3,...,4,4,4,4,4,4,4,4,4,4
10510,15196386,37,5.771597,0,0,0,0,0,1,2,...,4,4,4,4,4,4,4,4,4,4
11937,18857859,53,1.334769,0,0,0,0,0,1,1,...,4,4,4,4,4,4,4,4,4,4
11753,17794478,80,1.015104,0,0,0,0,0,10,3,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033,11660627,82,1.800660,0,0,0,0,1,3,3,...,9,9,9,9,9,9,9,9,9,9
4006,14252842,66,6.241771,0,0,0,0,0,1,3,...,4,4,4,4,4,4,4,4,4,4
657,14183192,50,24.440116,0,0,0,0,1,10,2,...,9,9,9,1,9,9,9,0,9,1
2351,16314070,23,1.926343,0,0,0,0,1,10,1,...,0,9,9,1,9,0,9,9,0,9


In [115]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier()

In [116]:
rf_pred = rf.predict(x_test)

In [117]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion = 'entropy', max_depth = 4,random_state=0)
dt.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=0)

In [118]:
dt_pred = dt.predict(x_test)

In [119]:
error=0
for i,j in zip(rf_pred,y_test['hospDIED']):
    if(i!=j):
        error+=1
print(error)

798


In [120]:
error=0
for i,j in zip(dt_pred,y_test['hospDIED']):
    if(i!=j):
        error+=1
print(error)

861


In [121]:
import sklearn.metrics as metrics
print("Accuracy:",metrics.accuracy_score(y_test, rf_pred))

Accuracy: 0.8920600568104964


In [122]:
import sklearn.metrics as metrics
print("Accuracy:",metrics.accuracy_score(y_test, dt_pred))

Accuracy: 0.8835384823481672


In [123]:
x_test = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/x_test.csv')
x_test = x_test.drop(['indextime'], axis=1)

In [124]:
y_pred = dt.predict(x_test)

In [ ]:
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

dot_data = StringIO()
export_graphviz(dt, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = x_test.columns,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('106062304_visualization.png')
Image(graph.create_png())

In [125]:
len(x_test)

2738

In [126]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

To export your predcition as a CSV file and hand in the CSV on elearn

In [44]:
output_path = STUDENT_ID + '_prediction.csv'

tree_test_pred = pd.DataFrame( {'subject_id': x_test.subject_id,
                  'prediction': y_pred } )
tree_test_pred.to_csv(output_path, index = False)

## Visualizing the Decision Tree *(10% bonus)*

**Note:** Save the visualization result image as **[STUDENT_ID]_visualization.png**

*   Your visualization image of the decision tree can contain **five** layers at most.


In [ ]:
#Decision Tree Visualization

# Save the Code File
Please save your code and submit it as an ipynb file! (**[STUDENT_ID]_hw2.ipynb**)